In [1]:
from codecarbon import EmissionsTracker
tracker = EmissionsTracker()
tracker.start()

[codecarbon INFO @ 13:39:24] [setup] RAM Tracking...
[codecarbon INFO @ 13:39:24] [setup] CPU Tracking...
[codecarbon WARNING @ 13:39:27] No CPU tracking mode found. Falling back on CPU constant mode. 
 Mac OS and ARM processor detected: Please enable PowerMetrics sudo to measure CPU

[codecarbon INFO @ 13:39:28] CPU Model on constant consumption mode: Apple M1
[codecarbon INFO @ 13:39:28] [setup] GPU Tracking...
[codecarbon INFO @ 13:39:28] No GPU found.
[codecarbon INFO @ 13:39:28] >>> Tracker's metadata:
[codecarbon INFO @ 13:39:28]   Platform system: macOS-10.16-x86_64-i386-64bit
[codecarbon INFO @ 13:39:28]   Python version: 3.9.12
[codecarbon INFO @ 13:39:28]   CodeCarbon version: 2.8.3
[codecarbon INFO @ 13:39:28]   Available RAM : 8.000 GB
[codecarbon INFO @ 13:39:28]   CPU count: 8
[codecarbon INFO @ 13:39:28]   CPU model: Apple M1
[codecarbon INFO @ 13:39:28]   GPU count: None
[codecarbon INFO @ 13:39:28]   GPU model: None
[codecarbon INFO @ 13:39:32] Saving emissions data to

In [2]:
import pandas as pd

# Cargar el dataset
df = pd.read_csv("../training.csv") 

# Mostrar nombres de columnas
print(df.columns.tolist())
import numpy as np
# Eliminar columnas que no aportan al modelo
df.drop(columns=["EventId", "Weight"], inplace=True)

# Convertir la variable objetivo a binaria: 's' = 1, 'b' = 0
df['Label'] = df['Label'].map({'s': 1, 'b': 0})

# Reemplazar los valores -999.0 por NaN
df.replace(-999.0, np.nan, inplace=True)

print("Dimensiones:", df.shape)
print("Valores nulos por columna:\n", df.isna().sum())

['EventId', 'DER_mass_MMC', 'DER_mass_transverse_met_lep', 'DER_mass_vis', 'DER_pt_h', 'DER_deltaeta_jet_jet', 'DER_mass_jet_jet', 'DER_prodeta_jet_jet', 'DER_deltar_tau_lep', 'DER_pt_tot', 'DER_sum_pt', 'DER_pt_ratio_lep_tau', 'DER_met_phi_centrality', 'DER_lep_eta_centrality', 'PRI_tau_pt', 'PRI_tau_eta', 'PRI_tau_phi', 'PRI_lep_pt', 'PRI_lep_eta', 'PRI_lep_phi', 'PRI_met', 'PRI_met_phi', 'PRI_met_sumet', 'PRI_jet_num', 'PRI_jet_leading_pt', 'PRI_jet_leading_eta', 'PRI_jet_leading_phi', 'PRI_jet_subleading_pt', 'PRI_jet_subleading_eta', 'PRI_jet_subleading_phi', 'PRI_jet_all_pt', 'Weight', 'Label']
Dimensiones: (250000, 31)
Valores nulos por columna:
 DER_mass_MMC                    38114
DER_mass_transverse_met_lep         0
DER_mass_vis                        0
DER_pt_h                            0
DER_deltaeta_jet_jet           177457
DER_mass_jet_jet               177457
DER_prodeta_jet_jet            177457
DER_deltar_tau_lep                  0
DER_pt_tot                        

In [3]:
# Imputar valores nulos con la mediana de cada columna
df.fillna(df.median(), inplace=True)

# Verificar que no queden nulos
print(df.isna().sum().sum() > 0)


False


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Separar X (features) e y (target)
X = df.drop(columns=['Label'])
y = df['Label']

# Escalar X
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# División en train y test (opcional, pero útil si luego quieres evaluar sobre test final)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

print("Tamaño del conjunto de entrenamiento:", X_train.shape)
print("Tamaño del conjunto de test:", X_test.shape)


Tamaño del conjunto de entrenamiento: (200000, 30)
Tamaño del conjunto de test: (50000, 30)


In [5]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

# Definimos el clasificador base
mlp = MLPClassifier(max_iter=200, early_stopping=True, random_state=42)

param_grid = {
    'hidden_layer_sizes': [(64, 32), (100, 50), (128, 64)],
    'activation': ['relu', 'tanh'],
    'solver': ['adam'],
    'alpha': [0.0001, 0.001],
    'learning_rate': ['constant', 'adaptive']
}

# GridSearch con validación cruzada
grid_search = GridSearchCV(
    mlp,
    param_grid,
    scoring='accuracy',
    cv=3,
    verbose=2,
    n_jobs=-1
)

# Entrenamiento
grid_search.fit(X_train, y_train)

# Mostrar mejores resultados
print("Mejores hiperparámetros:", grid_search.best_params_)
print("Accuracy en train:", grid_search.best_score_)

Fitting 3 folds for each of 24 candidates, totalling 72 fits


[codecarbon INFO @ 13:39:47] Energy consumed for RAM : 0.000013 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 13:39:47] Energy consumed for all CPUs : 0.000021 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 13:39:47] 0.000033 kWh of electricity used since the beginning.


[CV] END activation=relu, alpha=0.0001, hidden_layer_sizes=(64, 32), learning_rate=constant, solver=adam; total time=  18.9s
[CV] END activation=relu, alpha=0.0001, hidden_layer_sizes=(64, 32), learning_rate=adaptive, solver=adam; total time=  19.1s


[codecarbon INFO @ 13:40:02] Energy consumed for RAM : 0.000025 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 13:40:02] Energy consumed for all CPUs : 0.000042 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 13:40:02] 0.000067 kWh of electricity used since the beginning.


[CV] END activation=relu, alpha=0.0001, hidden_layer_sizes=(64, 32), learning_rate=adaptive, solver=adam; total time=  34.4s
[CV] END activation=relu, alpha=0.0001, hidden_layer_sizes=(64, 32), learning_rate=constant, solver=adam; total time=  34.7s
[CV] END activation=relu, alpha=0.0001, hidden_layer_sizes=(100, 50), learning_rate=constant, solver=adam; total time=  37.2s


...

[CV] END activation=tanh, alpha=0.001, hidden_layer_sizes=(128, 64), learning_rate=constant, solver=adam; total time= 1.4min


[codecarbon INFO @ 13:49:02] Energy consumed for RAM : 0.000474 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 13:49:02] Energy consumed for all CPUs : 0.000791 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 13:49:02] 0.001265 kWh of electricity used since the beginning.


[CV] END activation=tanh, alpha=0.001, hidden_layer_sizes=(128, 64), learning_rate=adaptive, solver=adam; total time=  59.7s
[CV] END activation=tanh, alpha=0.001, hidden_layer_sizes=(128, 64), learning_rate=adaptive, solver=adam; total time= 1.2min


[codecarbon INFO @ 13:49:17] Energy consumed for RAM : 0.000486 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 13:49:17] Energy consumed for all CPUs : 0.000812 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 13:49:17] 0.001298 kWh of electricity used since the beginning.
[codecarbon INFO @ 13:49:32] Energy consumed for RAM : 0.000499 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 13:49:32] Energy consumed for all CPUs : 0.000833 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 13:49:32] 0.001332 kWh of electricity used since the beginning.
[codecarbon INFO @ 13:49:32] 0.000386 g.CO2eq/s mean an estimation of 12.18222279015827 kg.CO2eq/year
[codecarbon INFO @ 13:49:47] Energy consumed for RAM : 0.000511 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 13:49:47] Energy consumed for all CPUs : 0.000854 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 13:49:47] 0.001365 kWh of electricity used since the beginning.
[codecarbon INFO @ 13:50:02] Energy consumed for RAM : 0.000524 kWh. RAM Power : 3.0 W
[codeca

Mejores hiperparámetros: {'activation': 'tanh', 'alpha': 0.001, 'hidden_layer_sizes': (128, 64), 'learning_rate': 'constant', 'solver': 'adam'}
Accuracy en train: 0.8372299993418583


In [6]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report

# Predecimos con el mejor modelo
y_pred = grid_search.best_estimator_.predict(X_test)
y_proba = grid_search.best_estimator_.predict_proba(X_test)[:, 1]

# Métricas
print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))
print("AUC-ROC:", roc_auc_score(y_test, y_proba))


print("\nClassification report:\n", classification_report(y_test, y_pred))

Accuracy: 0.83896
F1 Score: 0.7552732356695642
AUC-ROC: 0.908040039412041

Classification report:
               precision    recall  f1-score   support

           0       0.87      0.89      0.88     33065
           1       0.78      0.73      0.76     16935

    accuracy                           0.84     50000
   macro avg       0.82      0.81      0.82     50000
weighted avg       0.84      0.84      0.84     50000



Buenos resultados, pero intentamos conseguir un modelo aún mejor

In [7]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

# Nuevo clasificador base
mlp = MLPClassifier(max_iter=200, early_stopping=True, random_state=42)

# Nueva rejilla más ambiciosa
param_grid = {
    'hidden_layer_sizes': [(128, 64, 32), (256, 128, 64)],
    'activation': ['tanh'],
    'solver': ['adam'],
    'alpha': [0.0005, 0.001, 0.005],
    'learning_rate': ['adaptive']
}

# GridSearch con validación cruzada
grid_search_deep = GridSearchCV(
    mlp,
    param_grid,
    scoring='accuracy',
    cv=3,
    verbose=2,
    n_jobs=-1
)

# Entrenamiento
grid_search_deep.fit(X_train, y_train)

# Resultados
print("Mejores hiperparámetros:", grid_search_deep.best_params_)
print("Accuracy en train:", grid_search_deep.best_score_)


Fitting 3 folds for each of 6 candidates, totalling 18 fits


[codecarbon INFO @ 13:50:47] Energy consumed for RAM : 0.000561 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 13:50:47] Energy consumed for all CPUs : 0.000937 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 13:50:47] 0.001498 kWh of electricity used since the beginning.
[codecarbon INFO @ 13:51:02] Energy consumed for RAM : 0.000574 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 13:51:02] Energy consumed for all CPUs : 0.000958 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 13:51:02] 0.001532 kWh of electricity used since the beginning.
[codecarbon INFO @ 13:51:17] Energy consumed for RAM : 0.000586 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 13:51:17] Energy consumed for all CPUs : 0.000979 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 13:51:17] 0.001565 kWh of electricity used since the beginning.


[CV] END activation=tanh, alpha=0.0005, hidden_layer_sizes=(128, 64, 32), learning_rate=adaptive, solver=adam; total time=  55.2s


[codecarbon INFO @ 13:51:32] Energy consumed for RAM : 0.000599 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 13:51:32] Energy consumed for all CPUs : 0.000999 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 13:51:32] 0.001598 kWh of electricity used since the beginning.
[codecarbon INFO @ 13:51:32] 0.000386 g.CO2eq/s mean an estimation of 12.180090389988226 kg.CO2eq/year
[codecarbon INFO @ 13:51:47] Energy consumed for RAM : 0.000611 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 13:51:47] Energy consumed for all CPUs : 0.001020 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 13:51:48] 0.001632 kWh of electricity used since the beginning.
[codecarbon INFO @ 13:52:02] Energy consumed for RAM : 0.000624 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 13:52:02] Energy consumed for all CPUs : 0.001041 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 13:52:02] 0.001665 kWh of electricity used since the beginning.


[CV] END activation=tanh, alpha=0.001, hidden_layer_sizes=(128, 64, 32), learning_rate=adaptive, solver=adam; total time= 1.5min


...

[CV] END activation=tanh, alpha=0.005, hidden_layer_sizes=(256, 128, 64), learning_rate=adaptive, solver=adam; total time= 2.6min


[codecarbon INFO @ 13:57:03] Energy consumed for RAM : 0.000874 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 13:57:03] Energy consumed for all CPUs : 0.001458 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 13:57:03] 0.002331 kWh of electricity used since the beginning.


[CV] END activation=tanh, alpha=0.005, hidden_layer_sizes=(256, 128, 64), learning_rate=adaptive, solver=adam; total time= 2.6min


[codecarbon INFO @ 13:57:18] Energy consumed for RAM : 0.000886 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 13:57:18] Energy consumed for all CPUs : 0.001479 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 13:57:18] 0.002365 kWh of electricity used since the beginning.
[codecarbon INFO @ 13:57:33] Energy consumed for RAM : 0.000899 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 13:57:33] Energy consumed for all CPUs : 0.001499 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 13:57:33] 0.002398 kWh of electricity used since the beginning.
[codecarbon INFO @ 13:57:33] 0.000387 g.CO2eq/s mean an estimation of 12.194382683520086 kg.CO2eq/year
[codecarbon INFO @ 13:57:48] Energy consumed for RAM : 0.000911 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 13:57:48] Energy consumed for all CPUs : 0.001520 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 13:57:48] 0.002431 kWh of electricity used since the beginning.
[codecarbon INFO @ 13:58:03] Energy consumed for RAM : 0.000924 kWh. RAM Power : 3.0 W
[codec

Mejores hiperparámetros: {'activation': 'tanh', 'alpha': 0.001, 'hidden_layer_sizes': (128, 64, 32), 'learning_rate': 'adaptive', 'solver': 'adam'}
Accuracy en train: 0.8363799945167916


In [8]:
# Evaluación final en test con el nuevo modelo
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report

y_pred = grid_search_deep.best_estimator_.predict(X_test)
y_proba = grid_search_deep.best_estimator_.predict_proba(X_test)[:, 1]

print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))
print("AUC-ROC:", roc_auc_score(y_test, y_proba))
print("\nClassification report:\n", classification_report(y_test, y_pred))


Accuracy: 0.83632
F1 Score: 0.7547497752472281
AUC-ROC: 0.9088581129465092

Classification report:
               precision    recall  f1-score   support

           0       0.87      0.88      0.88     33065
           1       0.77      0.74      0.75     16935

    accuracy                           0.84     50000
   macro avg       0.82      0.81      0.82     50000
weighted avg       0.84      0.84      0.84     50000



### Selección del modelo MLP final

Tras comparar varias arquitecturas, seleccionamos como modelo final el MLP con configuración `(128, 64)`, función de activación `'tanh'` y tasa de aprendizaje `'constant'`. 

Este modelo ofrece el mejor equilibrio entre precisión, capacidad de generalización y simplicidad.

Aunque una arquitectura más profunda con `learning_rate='adaptive'` mejoró ligeramente el recall, no aportó mejoras significativas en el rendimiento global. Por tanto, priorizamos el modelo más eficiente y robusto.


In [9]:
emissions = tracker.stop()

[codecarbon INFO @ 13:58:43] Energy consumed for RAM : 0.000957 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 13:58:43] Energy consumed for all CPUs : 0.001597 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 13:58:43] 0.002554 kWh of electricity used since the beginning.
